In [2]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda, Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 64
IMG_HEIGHT = 64
IMG_CHANNELS = 25
TRAIN_PATH = 'GeneratedData/train/'
TEST_PATH = 'GeneratedData/test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
# Get train and test ids
train_ids = np.loadtxt(TRAIN_PATH + 'Directory.txt').astype(int)
test_ids = np.loadtxt(TEST_PATH + 'Directory.txt').astype(int)

In [ ]:
print("train ids(%i):" % (len(train_ids)), train_ids, "\n")
print("test ids(%i):" % (len(test_ids)), test_ids)

# Data processing

In [ ]:
X_train = np.zeros((len(train_ids), IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

# Get train images and masks
print('Getting train images and masks ... ')
sys.stdout.flush()
for n, id in tqdm(enumerate(train_ids), total=len(train_ids)):
    img_path = TRAIN_PATH + 'ImageMap-' + str(id) + '.dat'
    img_zxy = np.fromfile(img_path, dtype='uint8')
    img_zxy.shape = (IMG_CHANNELS, IMG_WIDTH, IMG_HEIGHT)
    # convert from 25x64x64 to 64x64x25
    img_xyz = np.zeros((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))
    for i in range(IMG_CHANNELS):
        img_xyz[:,:,i] = img_zxy[i,:,:]
    X_train[n] = img_xyz
    
    mask_path = TRAIN_PATH + 'LabelMap-' + str(id) + '.dat'
    mask = np.fromfile(mask_path, dtype=np.bool)
    mask.shape = (IMG_WIDTH, IMG_HEIGHT, 1)
    Y_train[n] = mask

# Get test images and masks
print('Getting test images and masks ... ')
sys.stdout.flush()
X_test = np.zeros((len(test_ids), IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
for n, id in tqdm(enumerate(test_ids), total=len(test_ids)):
    img_path = TEST_PATH + 'ImageMap-' + str(id) + '.dat'
    img_zxy = np.fromfile(img_path, dtype='uint8')
    img_zxy.shape = (IMG_CHANNELS, IMG_WIDTH, IMG_HEIGHT)
    # convert from 25x64x64 to 64x64x25
    img_xyz = np.zeros((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))
    for i in range(IMG_CHANNELS):
        img_xyz[:,:,i] = img_zxy[i,:,:]
    X_test[n] = img_xyz
    
    mask_path = TEST_PATH + 'LabelMap-' + str(id) + '.dat'
    mask = np.fromfile(mask_path, dtype=np.bool)
    mask.shape = (IMG_WIDTH, IMG_HEIGHT, 1)
    Y_test[n] = mask

print("Done!")

In [ ]:
# Check if training data looks okay
img_idx = random.randint(0, len(train_ids)-1)
color_idx = random.randint(0, IMG_CHANNELS-1)
imshow(X_train[img_idx,:,:,color_idx])
plt.show()
imshow(np.squeeze(Y_train[img_idx]))
plt.show()

# Create model

Next we build our U-Net model, loosely based on [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/pdf/1505.04597.pdf) and very similar to [this repo](https://github.com/jocicmarko/ultrasound-nerve-segmentation) from the Kaggle Ultrasound Nerve Segmentation competition.

![](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)

In [ ]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

# Normalize inputs
s = Lambda(lambda x: x / 255) (inputs)
# s = inputs

# Encoder layers
conv0 = Conv2D(4, (3, 3), activation='relu', padding='same') (s)
conv0 = Dropout(0.2)(conv0)
conv0 = Conv2D(4, (3, 3), activation='relu', padding='same') (conv0)
pool0 = MaxPooling2D((2, 2)) (conv0)

conv1 = Conv2D(8, (3, 3), activation='relu', padding='same') (pool0)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(8, (3, 3), activation='relu', padding='same') (conv1)
pool1 = MaxPooling2D((2, 2)) (conv1)

conv2 = Conv2D(16, (3, 3), activation='relu', padding='same') (pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(16, (3, 3), activation='relu', padding='same') (conv2)
pool2 = MaxPooling2D((2, 2)) (conv2)

conv3 = Conv2D(32, (3, 3), activation='relu', padding='same') (pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(32, (3, 3), activation='relu', padding='same') (conv3)
pool3 = MaxPooling2D((2, 2)) (conv3)

conv4 = Conv2D(64, (3, 3), activation='relu', padding='same') (pool3)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same') (conv4)

# Decoder layers
upconv7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (conv4)
upconv7 = concatenate([upconv7, conv3])
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same') (upconv7)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same') (conv7)

upconv8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (conv7)
upconv8 = concatenate([upconv8, conv2])
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same') (upconv8)
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same') (conv8)

upconv9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (conv8)
upconv9 = concatenate([upconv9, conv1])
conv9 = Conv2D(8, (3, 3), activation='relu', padding='same') (upconv9)
conv9 = Conv2D(8, (3, 3), activation='relu', padding='same') (conv9)

upconv10 = Conv2DTranspose(4, (2, 2), strides=(2, 2), padding='same') (conv9)
upconv10 = concatenate([upconv10, conv0], axis=3)
conv10 = Conv2D(4, (3, 3), activation='relu', padding='same') (upconv10)
conv10 = Conv2D(4, (3, 3), activation='relu', padding='same') (conv10)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv10)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# Training

Next we fit the model on the training data, using a validation split of 0.1. We use a small batch size because we have so little data. I recommend using checkpointing and early stopping when training your model. I won't do it here to make things a bit more reproducible (although it's very likely that your results will be different anyway).

In [ ]:
# Fit model
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-SonarNet-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=8, epochs=30, 
                   callbacks=[earlystopper, checkpointer])

In [ ]:
# Plot training & validation accuracy values
plt.plot(results.history['acc'])
plt.plot(results.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Prediction

In [ ]:
# Predict on train, val and test
model = load_model('model-SonarNet-1.h5', custom_objects={})

preds_train = model.predict(X_train[:int(X_train.shape[0]*0.75)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.75):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Detection

In [ ]:
# Perform a sanity check on some random training samples
img_idx = random.randint(0, len(preds_train_t)-1)
color_idx = random.randint(0, IMG_CHANNELS-1)

imshow(X_train[img_idx,:,:,color_idx])
plt.title("Image (1 color plane)")
plt.show()
plt.title("True mask")
imshow(np.squeeze(Y_train[img_idx]))
plt.show()
plt.title("Predicted mask")
imshow(np.squeeze(preds_train_t[img_idx]))
plt.show()

In [ ]:
# Perform a sanity check on some random validation samples
img_idx = random.randint(0, len(preds_val_t)-1)
color_idx = random.randint(0, IMG_CHANNELS-1)

imshow(X_train[img_idx,:,:,color_idx])
plt.title("Image (1 color plane)")
plt.show()
plt.title("True mask")
imshow(np.squeeze(Y_train[img_idx]))
plt.show()
plt.title("Predicted mask")
imshow(np.squeeze(preds_train_t[img_idx]))
plt.show()

# Evaluate

In [ ]:
# Evalute on test data
loss, accuracy = model.evaluate(X_test, Y_test)

In [ ]:
print("Loss: ", loss)
print("Accuracy: ", accuracy)

---

# Test code

In [ ]:
# show all images
for img in X_train:
    imshow(img[:,:,15])
    plt.show()

In [ ]:
# show all masks
for mask in Y_train:
    imshow(np.squeeze(mask))
    plt.show()